In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('raw_data2.csv', encoding='utf-8-sig', index_col=0)
df.head()

,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros
0,서울,4,3,2,180908204,"2,036",9516,7781959,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-복지의 끝판왕-매출 1조 클럽...
1,서울,4,3,3,60628188,"5,415",2858,1297880,성장판 열린 회사-복지의 끝판왕-HOT한 인기기업-보너스로 행복해요♥-장기근속수당제...
2,서울,4,1,3,60628188,"5,415",-,1297880,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-복지의 ...
3,서울,0,1,3,60628188,"5,415",-,1297880,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-매출 1조 클럽-HOT한 인...
4,서울,0,3,3,32879127,"3,241",5739,208129,성장판 열린 회사-채용에 적극적인 회사-HOT한 인기기업-지방에 위치한 대기업


In [4]:
tags_encoded = pd.read_csv('tags_encoded.csv', encoding='utf-8-sig', index_col=0)
# tags_encoded.head()
tags_score = dict(zip(tags_encoded['tags'], tags_encoded['score']))
tags_score

{'안정적인 장수 회사': 5,
 '성장판 열린 회사': 3,
 '구직자 인기기업': 5,
 '채용에 적극적인 회사': 3,
 '복지의 끝판왕': 1,
 '매출 1조 클럽': 5,
 '매출 상위 10%': 5,
 'HOT한 인기기업': 5,
 '아침을 왜 굶어요?': 1,
 '평균연봉 6천이상': 3,
 '신입연봉이 4천이상': 3,
 '스타트업 초봉 3,500만원↗': 3,
 '지방에 위치한 대기업': 5,
 '자기계발비 지원 기업': 1,
 '보너스로 행복해요♥': 1,
 '내 출퇴근은 내 맘대로': 1,
 '아낌없이 주는 커피': 1,
 '아이와 함께 출근해요': 1,
 '잘나가는 의료·바이오기업': 5,
 '주 4일 근무': 3,
 '집 걱정 안 하기': 3,
 '거침없이 성장하는 회사': 3,
 '매출 1000대 기업': 5,
 '인크루트 인증 우수기업': 3,
 '대기업 in 서울': 5,
 '공기업 in 서울': 5,
 '장기근속수당제공 기업': 3,
 '육아휴직 제공 기업': 1,
 'KOSPI 상장': 5,
 'KOSDAQ 상장': 5,
 '빵빵한 구내식당': 1,
 '휴식까지 챙겨줘요!': 1,
 'KONEX 상장': 3}

In [5]:
def encodeTagsByScore(tags: str):
    if isinstance(tags, float): return tags
    total_score = 0
    for tag in tags.split('-'):
        if tag !=  '':
            total_score += tags_score[tag]
    return total_score

df['pros_encoded'] = df['pros'].apply(encodeTagsByScore)
df['pros_encoded']
# df.head()

0      50.0
1      18.0
2      49.0
3      29.0
4      16.0
       ... 
165    28.0
166    36.0
167    36.0
168    36.0
169    36.0
Name: pros_encoded, Length: 170, dtype: float64

In [9]:
df_baseline = df.drop(columns=['pros'])
df_baseline.to_csv('data_baseline.csv', encoding='utf-8-sig')

### Data_basetline.csv
해야 될 전처리 작업들
- 결측값 처리 (회사 연 매출, 근로자수, 평균월급 회사자본금)  
중소기업에 유난히 결측값이 많은 것 같아서 결측값을 그대로 두는 방향으로 생각중.

클러스터링 : 결측값 처리 안해도 무관
회귀 : xgboost classifier -> decision tree 기반이므로 결측값 처리 안해도 무관

- region 범주형 데이터 처리  
수도권, 수도권외 광역시, 그 외 지방 지역으로 묶은 후 원-핫 인코딩 

- 각종 피쳐들 스케일링

### 전처리 작업 순서

결측값 처리 -> 피쳐 스케일링 -> region 인코딩  

In [9]:
df_temp = pd.read_csv('raw_data_temp-서울2부산340.csv', encoding='utf-8-sig')
df_temp = df_temp.dropna()
print(df_temp.isna().any())
df_temp.info


Unnamed: 0    False
region        False
cpname        False
title         False
career        False
education     False
jobtype       False
cptype        False
sales         False
employees     False
aversalary    False
capital       False
pros          False
dtype: bool


<bound method DataFrame.info of      Unnamed: 0 region                             cpname  \
0             0     서울                            (주)유한양행   
1             1     서울                               공군본부   
2             2     서울                           국민건강보험공단   
3             3     서울                        신용협동조합 공동채용   
4             4     서울                           코레일테크(주)   
..          ...    ...                                ...   
332         332     부산                          삼성생명보험(주)   
333         333     부산                          삼성생명보험(주)   
334         334     부산                          한국환경기계(주)   
338         338     부산  주식회사 라이프워시퍼 (LIFE WORSHIPER Inc.)   
339         339     부산                           (주)코디하우스   

                                               title       career education  \
0                                   2024 각 부문별 사원 모집     신입/경력1년↑      대졸이상   
1                  2024년도 공군 주관 일반군무원&장애인 구분모집 채용 공고  신입/경력(연차무관)      학력무관  